In [ ]:
#Ahmed Samir
#Mostafa Abdelraheem Mostafa
#Basmala mohamed gamal eldin
#Team Name: AMB

In [ ]:
import os
import numpy as np
import pandas as pd

os.environ['KAGGLE_USERNAME'] = 'ahmedsamir1598'
os.environ['KAGGLE_KEY'] = 'a1ea546319c78fa3d825b347704b684d'

In [ ]:
!kaggle competitions download -c gdsc-ml-workshop-final-project

In [ ]:
import zipfile
import os

# Define the path to the zip file
zip_file_path = '/kaggle/working/gdsc-ml-workshop-final-project.zip'

# Define the directory to extract to
extract_dir = '/kaggle/working/'

# Ensure the directory exists
os.makedirs(extract_dir, exist_ok=True)

# Extract the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("Extraction completed!")

In [ ]:
train_df = pd.read_csv("/kaggle/working/train.csv")
test_df = pd.read_csv("/kaggle/working/test.csv")

In [ ]:
print(train_df.info())
print(train_df.describe())
print(train_df.isnull())

In [ ]:
print(test_df.info())
print(test_df.describe())
print(test_df.isnull())

In [ ]:
train_df = train_df.dropna()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set_style('whitegrid')

In [ ]:
print(train_df)

In [ ]:
print(train_df['subject'].unique())
print(train_df['subject'].nunique())

***It would be better to reduce the num of categories in subject, and try feature engineerning by adding another feature "Region" that will include "worldnews", "US_News", "Middle-east".***

In [ ]:
train_df['subject'] = train_df['subject'].apply(lambda x: 'politics' if x == 'politicsNews' else x)

# Verify the changes
print(train_df['subject'].value_counts())
print(train_df['subject'].nunique())

In [ ]:
common_classes = [
    'politics', 'worldnews', 'News', 'left-news', 
    'Government News', 'US_News', 'Middle-east'
]

# Function to classify subjects
def classify_subject(subject):
    if subject in common_classes:
        return subject
    else:
        return 'longnews'

# Apply the classification function to create a new column
train_df['subject_class'] = train_df['subject'].apply(classify_subject)
print(train_df['subject_class'].value_counts())
print(train_df['subject_class'].nunique())


Something is suspicious about worldnews as a subject. I believe it might contain also be categorizing data for US and Middle east.

Found entry with such idea

9540 ISMAILIA,Egypt (Reuters) - At least 24 militants and six soldiers were killed on Sunday in attacks o...
worldnews

9658
A house divided: How Saudi Crown Prince purged royal family rivals
BEIRUT/RIYADH (Reuters) - The first hint that something was amiss came in a letter. On Saturday Nov....
worldnews

40398
U.S. judge rejects bid to dismiss Indonesian immigrant's lawsuit
BOSTON (Reuters) - A federal judge on Wednesday declined to dismiss a lawsuit by an Indonesian illeg...
worldnews

In [ ]:
train_df.dropna(inplace=True)

In [ ]:
def assign_region(title):
    title_lower = title.lower()
    if any(keyword in title_lower for keyword in ['egypt', 'saudi', 'middle_east', 'iraq', 'palestine', 'iran', 'jordan', 'yemen', 'uae', 'qatar', 'lebanon', 'israel', 'kuwait','oman', 'syria', 'bahrain', 'türkiye', 'cyprus', 'cairo','kurds','mena' ]):
        return 'Middle_East'
    elif any(keyword in title_lower for keyword in ['us', 'u.s.', 'united states']):
        return 'US'
    elif any(keyword in title_lower for keyword in ['swiss', 'switzerland','croatia', 'germany','berlin', 'france', 'italy', 'spain', 'uk', 'united kingdom', ]):
        return 'Europe'
    elif any(keyword in title_lower for keyword in ['china', 'japan', 'korea','myanmar','rohingya','india','russia']):
        return 'Asia'
    elif any(keyword in title_lower for keyword in ['nigeria', 'kenya', 'south africa','cameroon','sudan','ethiopia','somalia']):
        return 'Africa'
    elif any(keyword in title_lower for keyword in ['brazil', 'argentina', 'colombia']):
        return 'South_America'
    elif any(keyword in title_lower for keyword in ['zealand', 'australia','nz']):
        return 'Australia'

    else:
        return 'WorldNews'

# Assuming df_no_duplicates_text_and_title is your DataFrame
train_df['region'] = train_df['title'].apply(assign_region)
print(train_df)

In [ ]:
print(train_df['region'].value_counts())
print(train_df['region'].nunique())

In [ ]:
print(train_df['date'])

In [ ]:
def safe_to_datetime(date):
    try:
        return pd.to_datetime(date, errors='raise')
    except Exception as e:
        print(f"Date parsing error for date '{date}': {e}")
        return pd.NaT

# Apply date parsing with error handling
train_df['date'] = train_df['date'].apply(safe_to_datetime)

# Drop rows where date parsing failed (i.e., date is NaT)
#df_no_duplicates_text_and_title.dropna(subset=['date'], inplace=True)

# Define date ranges
date_ranges = [
    (pd.Timestamp('2015-03-31'), pd.Timestamp('2015-05-22')),
    (pd.Timestamp('2015-05-22'), pd.Timestamp('2015-07-14')),
    (pd.Timestamp('2015-07-14'), pd.Timestamp('2015-09-05')),
    (pd.Timestamp('2015-09-05'), pd.Timestamp('2015-12-20')),
    (pd.Timestamp('2015-12-20'), pd.Timestamp('2016-02-10')),
    (pd.Timestamp('2016-02-10'), pd.Timestamp('2016-04-03')),
    (pd.Timestamp('2016-04-03'), pd.Timestamp('2016-05-26')),
    (pd.Timestamp('2016-05-26'), pd.Timestamp('2016-07-18')),
    (pd.Timestamp('2016-07-18'), pd.Timestamp('2016-09-09')),
    (pd.Timestamp('2016-09-09'), pd.Timestamp('2016-10-31')),
    (pd.Timestamp('2016-10-31'), pd.Timestamp('2016-12-12')),
    (pd.Timestamp('2016-12-12'), pd.Timestamp('2017-02-14')),
    (pd.Timestamp('2017-02-14'), pd.Timestamp('2017-04-08')),
    (pd.Timestamp('2017-04-08'), pd.Timestamp('2017-05-31')),
    (pd.Timestamp('2017-05-31'), pd.Timestamp('2017-07-22')),
    (pd.Timestamp('2017-07-22'), pd.Timestamp('2017-09-13')),
    (pd.Timestamp('2017-09-13'), pd.Timestamp('2017-11-05')),
    (pd.Timestamp('2017-11-05'), pd.Timestamp('2017-12-28')),
    (pd.Timestamp('2017-12-28'), pd.Timestamp('2018-02-19'))
]

# Function to assign date range
def assign_date_range(date):
    for i, (start, end) in enumerate(date_ranges):
        if start <= date <= end:
            return f'Range_{i+1}'
    return 'ٌRange_0'

# Apply function to create new column
train_df['date_range'] = train_df['date'].apply(assign_date_range)

# Print the result
print(train_df[['date', 'date_range']])

In [ ]:
print(train_df['date_range'].value_counts())
print(train_df['date_range'].nunique())

In [ ]:
train_df.to_csv('output.csv', index=False)
print("DataFrame has been saved to 'output.csv'")

In [ ]:
!pip install spacy
!python -m spacy download en_core_web_sm


In [ ]:
train_df['cleaned_text'] = df_no_duplicates_text_and_title['cleaned_text']
train_df['cleaned_title']= df_no_duplicates_text_and_title['cleaned_title']

In [ ]:
print(train_df.head())

In [ ]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

nlp = spacy.load('en_core_web_sm')
def preprocess(text):
    # Apply the spaCy model to the text
    doc = nlp(text)
    
    # Remove stop words and punctuation, and lemmatize the words
    words = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    
    return ' '.join(words)
train_df['cleaned_text'] = train_df['text'].apply(preprocess)
train_df['cleaned_title'] = train_df['title'].apply(preprocess)


In [ ]:
train_df.to_csv('tokenized.csv', index=False)
print("DataFrame has been saved to 'tokenized.csv'")

In [ ]:
df = train_df
df['combined'] = df['cleaned_title'] + ' ' + df['cleaned_text']
df['combined']=df['combined'].astype(str)
from sklearn.model_selection import train_test_split
features = ['combined','date_range', 'subject_class', 'region']
X = df[features]
y = df['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
print(f'X_train shape after splitting: {X_train.shape}')
print(f'y_train shape after splitting: {len(y_train)}')
print(f'X_test shape after splitting: {X_test.shape}')
print(f'y_test shape after splitting: {len(y_test)}')


In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Define the columns to be used for different transformations
text_col = 'combined'
other_cols = ['date_range', 'subject_class', 'region']

# Define the preprocessing steps for different columns
preprocessor = ColumnTransformer(
    transformers=[
        ('text', TfidfVectorizer(max_features=5000), text_col),
        ('other', OneHotEncoder(), other_cols)
    ],
    remainder='passthrough'
)

# Define the model pipeline
model = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression())
])

# Fit the model
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)

print('Accuracy:', accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gbc = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', GradientBoostingClassifier(n_estimators=100, random_state=42))
])
gbc.fit(X_train, y_train)
y_gbc_pred = gbc.predict(X_test)
accuracy_gbc = accuracy_score(y_test, y_gbc_pred)
print(f"Model Accuracy: {accuracy_gbc:.2f}")
print(classification_report(y_test, y_gbc_pred))

In [ ]:
from xgboost import XGBClassifier

xgbc = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier(n_estimators=100, random_state=42))
])
xgbc.fit(X_train, y_train)
y_xgbc_pred = xgbc.predict(X_test)
accuracy_xgbc = accuracy_score(y_test, y_xgbc_pred)
print(f"Model Accuracy: {accuracy_xgbc:.2f}")
print(classification_report(y_test, y_xgbc_pred))

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier, StackingClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
estimators = [
    ('gb', make_pipeline(StandardScaler(), GradientBoostingClassifier(n_estimators=100,random_state=42))),
    ('xgb', make_pipeline(StandardScaler(), XGBClassifier(n_estimators=100, random_state=42)))
]

# Define stacking classifier with Logistic Regression as the meta-classifier
stacking_model = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(),
    passthrough=False,  # If passthrough=True, the original features are added to the meta-classifier
    cv=5  # Number of cross-validation folds
))
])

# Fit the stacking model
stacking_model.fit(X_train, y_train)

# Predict using the stacking model
stacking_preds = stacking_model.predict(X_test)

# Evaluate the model
print("Stacking Classifier Accuracy:", accuracy_score(y_test, stacking_preds))
print(classification_report(y_test, stacking_preds))

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# Assuming `test.csv` contains your test data
test_df = pd.read_csv("/kaggle/working/test.csv")
test_df['subject'] = test_df['subject'].apply(lambda x: 'politics' if x == 'politicsNews' else x)
test_df['region'] = test_df['title'].apply(assign_region)
test_df['date'] = test_df['date'].apply(safe_to_datetime)
test_df['date_range'] = test_df['date'].apply(assign_date_range)
test_df['cleaned_text'] = test_df['text'].apply(preprocess)
test_df['cleaned_title'] = test_df['title'].apply(preprocess)
test_df['combined'] = test_df['cleaned_title'] + ' ' + test_df['cleaned_text']
test_df['subject_class'] = test_df['subject'].apply(classify_subject)
features = ['combined','date_range', 'subject_class', 'region']
X_test_test = test_df[features]
y_pred = model.predict(X_test_test)
y_gbc_pred = gbc.predict(X_test_test)
y_xgbc_pred = xgbc.predict(X_test_test)
# Print the predictions
#print(y_pred)


In [ ]:
count_lr = 0
count_gbc = 0
count_xgb = 0
count_stacking = 0
for i in y_pred:
    if i == 0:
        count_lr+=1
for i in y_gbc_pred:
    if i == 0:
        count_gbc+=1
for i in y_xgbc_pred:
    if i == 0:
        count_xgb+=1
#for i in stacking_preds:
#    if i == 0:
#        count_stacking+=1
print(f"count0 LR:{count_lr}")
print(f"count0 count_gbc:{count_gbc}")
print(f"count0 count_xgb:{count_xgb}")
#print(f"count0 count_stacking:{count_stacking}")

In [ ]:
count_lr = 0
count_gbc = 0
count_xgb = 0
count_stacking = 0
for i in y_pred:
    if i == 1:
        count_lr+=1
for i in y_gbc_pred:
    if i == 1:
        count_gbc+=1
for i in y_xgbc_pred:
    if i == 1:
        count_xgb+=1
#for i in stacking_preds:
#    if i == 1:
#        count_stacking+=1
print(f"count1 LR:{count_lr}")
print(f"count1 count_gbc:{count_gbc}")
print(f"count1 count_xgb:{count_xgb}")
#print(f"count1 count_stacking:{count_stacking}")

In [ ]:
result_df = pd.DataFrame({'ID': test_df['ID'], 'TARGET': y_xgbc_pred})

# Save the DataFrame to a CSV file
result_df.to_csv('predictions.csv', index=False)

In [ ]:
result_df = pd.DataFrame({'ID': test_df['ID'], 'TARGET': y_gbc_pred})

# Save the DataFrame to a CSV file
result_df.to_csv('predictions_gbc.csv', index=False)

In [ ]:
predictions = pd.read_csv("/kaggle/working/predictions.csv")
print(predictions)